# Example Notebook to show how to predict solar power generation

Make sure to have installed the quart_solar_forecast package: 
```
pip install quartz-solar-forecast
```

## Setup

In [ ]:
# Import forecast script and PVSite class.
from quartz_solar_forecast.forecast import run_forecast
from quartz_solar_forecast.pydantic_models import PVSite
from quartz_solar_forecast.data import get_nwp
from datetime import datetime, timedelta

# Import matplotlib and plotly for plotting.
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# ignore future warnings
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

## Make Predictions

To make predictions, we first need to specify the location and the capacity of the panel, for which we want to make predictions. This is done using the class *PVSite*. The location is given in latitude between -90 and 90 and longitude between -180 and 180 degrees.

This site object is then used to make predictions for a specific time for 48 hours ahead. In the example, the date is chosen as the current time.

Note, that the prediction may take a couple of minutes.

In [ ]:
# make a pv site object
site = PVSite(latitude=51.75, longitude=-1.25, capacity_kwp=1.25)

# run model for today
predictions_df = await run_forecast(site)

The result is a pandas data frame with the solar energy predicted ("power_wh") for each time step in a 15 minute interval from the given start date on.

In [ ]:
predictions_df

We can plot the predictions.

In [ ]:
# Create an interactive plot of the forecast using plotly.
fig = px.line(predictions_df.reset_index().rename(columns={"index": "date"}),
              x="date", y="power_kw",
              labels={"power_kw": "Power (kw)"},
              title="Solar Energy Prediction")
fig.show()

Let's consider a second example for a start time further in the past.

In [ ]:
# Run the forecast for a specific initial timestamp.
# This generates a forecast at 15 minute intervals for 48 hours.
ts = datetime.today() - timedelta(weeks=3)
site = PVSite(latitude=51.75, longitude=-1.25, capacity_kwp=1.25)

predictions_df = await run_forecast(site, ts=ts)

In this case, since the requested date lies more than 3 months in the past, we get a notification that different meterological data was used to make the predictions.

In [ ]:
predictions_df

In [ ]:
# Create an interactive plot of the forecast using plotly.
fig = px.line(predictions_df.reset_index().rename(columns={"index": "date"}),
              x="date", y="power_kw",
              labels={"power_kw": "Power (kw)"},
              title="Solar Energy Prediction")
fig.show()

## Visualize meteorological input data

Additionally we can load and visualize the meteorological data used for the predictions. We can use the *get_nwp* function. This function returns the data as an *xarray*.

In [ ]:
met_data = get_nwp(site=site, ts=ts, nwp_source="icon")
met_data

The meterolocial variables used are:

```
variables = [
        "temperature_2m",
        "precipitation",
        "cloud_cover_low",
        "cloud_cover_mid",
        "cloud_cover_high",
        "wind_speed_10m",
        "shortwave_radiation",
        "direct_radiation"
    ]
```

The variable names in the returned array are abbreviated as:

```
xarray_variables = ["t",
                "prate",
                "lcc",
                "mcc",
                "hcc",
                "si10",
                "dswrf",
                "dlwrf"
  ]
```


Add the data to the predictions dataframe.

In [ ]:
variables = [
        "temperature_2m",
        "precipitation",
        "cloud_cover_low",
        "cloud_cover_mid",
        "cloud_cover_high",
        "wind_speed_10m",
        "shortwave_radiation",
        "direct_radiation"
    ]

for i, var in enumerate(variables):
  predictions_df[var] = met_data.icon.variable[:,i]

In [ ]:
predictions_df

In [ ]:
# reset index and rename index column to date
predictions_df = predictions_df.reset_index().rename(columns={"index": "date"})

In [ ]:
figures =  [px.line(predictions_df,
              x="date", y="power_kw"),
            px.line(predictions_df,
              x="date", y="temperature_2m"),
            px.line(predictions_df,
              x="date", y="precipitation"),
            px.line(predictions_df,
              x="date", y="cloud_cover_low"),
            px.line(predictions_df,
              x="date", y="cloud_cover_mid"),
            px.line(predictions_df,
              x="date", y="cloud_cover_high"),
            px.line(predictions_df,
              x="date", y="wind_speed_10m"),
            px.line(predictions_df,
              x="date", y="shortwave_radiation",
              labels={"shortwave_radiation": "Shortwave Radiation"}),
            px.line(predictions_df,
              x="date", y="direct_radiation")
    ]

fig = make_subplots(rows=len(figures), cols=1,
                    subplot_titles=("Power (kw)",
                                    "Temperature", "Precipitation",
                                    "Cloud Cover (Low)", "Cloud Cover (Mid)",
                                    "Cloud Cover (High)", "Wind Speed (10m)",
                                    "Shortwave Radiation", "Direct Radiation"))
fig.update_layout(height=1400, width=800, title_text="Meterological Variables")

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=i+1, col=1)


fig.show();

## Use the XGBoost model

In [ ]:
ts = datetime.today() - timedelta(weeks=3)
site = PVSite(latitude=51.75, longitude=-1.25, capacity_kwp=1.25)

predictions_df = await run_forecast(site, model="xgb", ts=ts)

In [ ]:
predictions_df

In [ ]:
# Create an interactive plot of the forecast using plotly.
fig = px.line(predictions_df.reset_index().rename(columns={"index": "date"}),
              x="date", y="power_kw",
              labels={"power_kw": "Power (kw)"},
              title="Solar Energy Prediction")
fig.show()